In [1]:
%pip install pyriksdagen
from lxml import etree
import progressbar
from pyparlaclarin.read import paragraph_iterator, speeches_with_name,parlaclarin_to_txt,parlaclarin_to_md
from pyriksdagen.utils import protocol_iterators, download_corpus
from pyriksdagen.metadata import load_Corpus_metadata
import pyriksdagen
# We need a parser for reading in XML data
parser = etree.XMLParser(remove_blank_text=True)
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re


Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --upgrade pyparlaclarin

In [ ]:
pip show pyparlaclarin

In [ ]:
print(pyriksdagen.__spec__)
download_corpus(partitions=["politicians", "records"])

In [2]:
protocols = list(protocol_iterators(corpus_root="data/", start=1867, end=202122))

In [ ]:
len(protocols)

In [ ]:
def oppna_data_to_dict(input_dict):
    """
    Load protocols with the new XML / HTML structure (from 2013 onwards)
    and convert it to a python dict with contents.
    """
    data = {}
    data["paragraphs"] = []

    # Metadata
    session = input_dict["dokumentstatus"]["dokument"]["rm"]
    session = session.replace("/", "")
    pid = input_dict["dokumentstatus"]["dokument"]["nummer"]
    date = input_dict["dokumentstatus"]["dokument"]["datum"]
    html = input_dict["dokumentstatus"]["dokument"]["html"]
    html_tree = clean_html(html)
    year = int(date.split("-")[0])
    protocol_id = f"prot-{session}--{pid}"

    data["protocol_id"] = protocol_id
    data["date"] = date.split(" ")[0]
    data["session"] = session

    # New HTML structure with div[@class='Section1']
    section1 = html_tree.xpath(".//div[@class='Section1']")
    for elements in section1:
        for elem in elements:
            if elem.tag in ["p", "h1", "h2"]:
                elemtext = "".join(elem.itertext())
                linebreak = elemtext.strip() == "" and "\n" in elemtext
                if linebreak:
                    pass
                else:
                    paragraph = elemtext.strip()
                    paragraph = paragraph.replace("\n", " ")
                    paragraph = re.sub("\\s+", " ", paragraph)
                    data["paragraphs"].append(paragraph)

    if len(data["paragraphs"]) == 0:
        tree = html_tree

        # Old data structure 1990-2003
        pres = tree.findall(".//pre")
        if len(pres) > 0:
            for pre in pres:
                if pre.text is not None:
                    tblocks = re.sub("([a-zß-ÿ,])- ?\n ?([a-zß-ÿ])", "\\1\\2", pre.text)
                    tblocks = re.sub("([a-zß-ÿ,]) ?\n ?([a-zß-ÿ])", "\\1 \\2", tblocks)
                    for paragraph in tblocks.split("\n"):
                        paragraph = paragraph.replace("\n", " ")
                        paragraph = paragraph.replace("\n", " ")
                        data["paragraphs"].append(paragraph)

        # Standard HTML structure, roughly 2003-2013
        elif len(tree.xpath("//div[@class='indrag']")) > 0:
            tree = tree.xpath("//body")[0]
            for elem in tree:
                elemtext = "".join(elem.itertext())
                linebreak = elemtext.strip() == "" and "\n" in elemtext
                if elem.tag == "br" or linebreak:
                    pass
                else:
                    paragraph = elemtext.strip()
                    paragraph = paragraph.replace("\n", " ")
                    paragraph = re.sub("\\s+", " ", paragraph)
                    data["paragraphs"].append(paragraph)
    return data


In [ ]:
protocol_in_question = protocols[10]
root = etree.parse(protocol_in_question, parser).getroot()

In [ ]:
protocol_in_question

In [ ]:

for elem in list(paragraph_iterator(root, output="lxml"))[:7]:
  print(" ".join(elem.itertext()))

In [ ]:
data_train=[]
data_test=[]
data_valid=[]

for i in range(len(protocols)):
  protocol_in_question = protocols[i]

  h=hash(protocol_in_question)
  h_adjusted = h % (2**32 - 1)
  np.random.seed(h_adjusted)
  r = np.random.rand()
  root = etree.parse(protocol_in_question, parser).getroot()
  element_str=paragraph_iterator(root, output="str")
  if r<=0.7:
    data_train.append({"protocole": protocol_in_question,"texte": " ".join(element_str)})
  elif r<=0.85:
    data_test.append({"protocole": protocol_in_question,"texte": " ".join(element_str)})
  else :
    data_valid.append({"protocole": protocol_in_question,"texte": " ".join(element_str)})

df_train=pd.DataFrame(data_train)
df_test=pd.DataFrame(data_test)
df_valid=pd.DataFrame(data_valid)

print(df_train.head())


In [ ]:
print(df.iloc[0]["texte"])

In [ ]:
df.to_pickle("swerick_data_long.pkl")

In [ ]:
from datasets import load_dataset
swerick_dataset = load_dataset("pandas",data_files="swerick_data.pkl")

In [ ]:
print(swerick_dataset)

In [ ]:
def train_test_valid(df):
    df_train=pd.DataFrame()
    df_test=pd.DataFrame(
    )
    df_valid=pd.DataFrame()

    for s in df["protocole"].unique():
        subset_df = df[df["protocole"] == s]
        h=hash(s)
        h_adjusted = h % (2**32 - 1)
        np.random.seed(h_adjusted)
        r = np.random.rand()
        if r <= 0.7 :
            df_train = pd.concat([df_train, subset_df], ignore_index=True)
        elif r <=0.85:
            df_test = pd.concat([df_test, subset_df], ignore_index=True)
        else:
            df_valid = pd.concat([df_valid, subset_df], ignore_index=True)
    return df_train,df_test,df_valid




In [ ]:
df_train,df_test,df_valid=train_test_valid(df)

In [ ]:
print(df_test)

In [ ]:
print(df_valid)

In [ ]:
print(df_train)



In [ ]:
#df_train,df_test = train_test_split(df,test_size=0.2,random_state=42)
df_train.to_pickle("swerick_data_random_train.pkl")
df_test.to_pickle("swerick_data_random_test.pkl")
df_valid.to_pickle("swerick_data_random_valid.pkl")

In [ ]:
#datasest
data_files = {"train": "swerick_data_train.pkl", "test": "swerick_data_test.pkl"}
swerick_dataset = load_dataset("pandas",data_files=data_files)
print(swerick_dataset)


In [ ]:
protocol_in_question

In [ ]:



notes=[]
intros=[]

for i in range(len(protocols)):
    protocol_in_question = protocols[i]
    with open(protocol_in_question,"r") as f:
        xml_content=f.read()
    soup=BeautifulSoup(xml_content,"xml")
    note_elements=soup.find_all("note")

    for note in note_elements:
        note_text=note.text.strip()
        notes.append(note_text)

        next_element=note.get("type")
        if next_element == "speaker" :
            intros.append(True)
        else :
            intros.append(False)
 
df=pd.DataFrame({"Note" :notes, "Intro":intros})

print(df)

In [ ]:
df_train_intro,df_test_intro = train_test_split(df,test_size=0.2,random_state=42)
df_train_intro.to_pickle("swerick_data_intro_train.pkl")
df_test_intro.to_pickle("swerick_data_intro_test.pkl")

In [ ]:
metadata =load_Corpus_metadata()
print(metadata)


In [ ]:
metadata =pd.DataFrame(metadata)
metadata=metadata.set_index("person_id")
metadata

In [ ]:
type(metadata.loc["i-Ddmtm1uG9esPH37c8XjUXZ" ,"party"].dropna())

In [ ]:
notes=[]
id=[]
party=[]
gender=[]
protocol=[]

for i in range(len(protocols)):
    protocol_in_question = protocols[i]
    with open(protocol_in_question,"r") as f:
        xml_content=f.read()
    soup=BeautifulSoup(xml_content,"xml")
    note_elements=soup.find_all("u")


    for note in note_elements:
        note_text = note.text.strip()
        note_text = re.sub(r'\s+',' ',note_text)
        note_text = re.sub(r'\n+',' ',note_text)
        next_element = note.get("who")
        if next_element != "unknown":
            id.append(next_element)
            notes.append(note_text)

            party_value = metadata.loc[next_element, "party"]
            if isinstance(party_value, pd.Series):
                party_values = party_value.dropna()
                if not party_values.empty:
                    party.append(party_values.iloc[0])
                else:
                    party.append(np.nan)
            else:
                party.append(party_value)

            gender_value = metadata.loc[next_element, "gender"]
            if isinstance(gender_value, pd.Series):
                gender_values = gender_value.dropna()
                if not gender_values.empty:
                    gender.append(gender_values.iloc[0])
                else:
                    gender.append(np.nan)
            else:
                gender.append(gender_value)

            protocol.append(protocol_in_question)

df = pd.DataFrame({"protocol": protocol, "Note": notes, "id": id, "party": party, "gender": gender})

        

In [8]:
#NOte Seg classification



text=[]
note_seg=[]

for i in range(len(protocols)):
    protocol_in_question = protocols[i]
    with open(protocol_in_question,"r") as f:
        xml_content=f.read()
    soup=BeautifulSoup(xml_content,"xml")
    note_elements=soup.find_all("note")
    seg_elements=soup.find_all("seg")

    for note in note_elements:
        text.append(note.text.strip())
        note_seg.append("note")

    for seg in seg_elements:
        text.append(seg.text.strip())
        note_seg.append("seg")

df=pd.DataFrame({"Text" :text, "Note/seg":note_seg})

print(df)

                                                       Text Note/seg
0         Sedan, i kraft af Rikets Regeringsform, lagtim...     note
1            Fredagen den 18 Januari 1867, kl. '/, 11 f. m.     note
2         Riksdagens Andra Kammare uti den för dess samm...     note
3                                              Mine Herrar!     note
4         Enligt 33 8 i Riksdags-ordningen skall, innan ...     note
...                                                     ...      ...
14108231  2021/22:1889 Sveriges arbete för kärnvapennedr...      seg
14108232  till näringsminister Karl-Petter Thorwaldsson (S)      seg
14108233  2021/22:1909 Växande skulder till följd av hög...      seg
14108234                                        GERGÖ KISCH      seg
14108235         Tryck: Elanders Sverige AB, Vällingby 2022      seg

[14108236 rows x 2 columns]


In [9]:
df_train_seg,df_test_seg = train_test_split(df,test_size=0.2,random_state=42)
df_train_seg.to_pickle("swerick_data_seg_train.pkl")
df_test_seg.to_pickle("swerick_data_seg_test.pkl")

df

In [ ]:
df.to_pickle("gender_party_dataset.pkl")

In [ ]:
df = df.rename(columns={"protocol": "protocole"})


In [ ]:
df.iloc[0]["Note"]

In [ ]:
df_train,df_test,df_valid=train_test_valid(df)

In [ ]:
df_train.iloc[0]["Note"]

In [ ]:
df_train.to_pickle("swerick_data_party_train.pkl")
df_test.to_pickle("swerick_data_party_test.pkl")
df_valid.to_pickle("swerick_data_party_valid.pkl")